# Exploratory Data Analysis - Notebook

# All in One Place: High Value Customer Identification

**All in one place** is an online retail store that sells second-hand products from various brands at lower prices. With just over a year of operation, the marketing team noticed that some customers from their base purchase more expensive products more frequently, contributing significantly to the company's revenue.

Based on this insight, the **marketing team** decided to **launch a loyalty program** for the **top customers** in their base, named Insiders.
Moreover, the Marketing teams lacks the necessary knowledge to identify the customers to join the program. As a result, this task have been assigned to the company's data team, whose should develop a solution and a report to respond the marketing queries below.
Who are the eligible individuals to participate in the program?
How many customers will be part of the group?
What are the main characteristics of these customers?
What percentage of the revenue comes from the selected group?

The company's data team, in accordance with the company's marketing team had combined that, as a first solution, they will use a RFM Matrix to identify the customers.


# 0 - IMPORTS

In [ ]:
# Data Maniputalion and Data Analysis
import re
import warnings

import pandas                as pd
import numpy                 as np
import seaborn               as sns
import plotly.express        as px
import plotly.graph_objects  as go
import matplotlib.cm         as cm
import matplotlib            as mpl

from plotly.offline          import iplot
from matplotlib              import pyplot          as plt


warnings.filterwarnings( 'ignore' )

## 0.1 - Helper Functions

In [ ]:
def drop_and_rename_duplicate_columns(df):
    """
    Remove duplicate columns resulting from a merge and renames columns to remove the '_x' suffix.
    Removes rows related to canceled or returned invoices: where the value of the `invoice_status_y` column is equal to 'True'.

    Parameters:
    - df (pd.DataFrame): DataFrame resulting from a merge with possible duplicate columns.

    Returns:
    - pd.DataFrame: DataFrame with duplicate columns removed and renamed.
    """

    # Checks if the 'invoice_status_y' column exists and filters where the value is True
    if 'invoice_cancelled_y' in df.columns:
        df = df[~df['invoice_cancelled_y'].fillna(False)]
    
    # Identifies columns with '_x' and '_y' suffixes after the merge
    duplicate_columns = [col for col in df.columns if col.endswith('_x') or col.endswith('_y')]
    
    # Creates a mapping to keep only one occurrence and rename columns
    cols_to_keep = {}
    for col in duplicate_columns:
        base_name = col[:-2]  # Removes the '_x' or '_y' suffix
        if base_name not in cols_to_keep:
            # Saves the column with the '_x' suffix to keep and rename
            cols_to_keep[base_name] = col

    # Defines the columns to drop, keeping only one of each duplicate pair
    cols_to_drop = set(duplicate_columns) - set(cols_to_keep.values())
    df = df.drop(columns=cols_to_drop)

    # Renames the remaining columns, removing the '_x' suffix
    df = df.rename(columns={old_name: base_name for base_name, old_name in cols_to_keep.items()})

    # Filters the columns, removing invoice_cancelled
    cols = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date','unit_price', 'customer_id', 'country']

    return df[cols]

# 1 - DATA MANIPULATION

## 1.1 - Loading Data

In [ ]:
df = pd.read_csv("../data/raw/Ecommerce.csv")

# drop extra column
df = df.drop( columns=['Unnamed: 8'], axis=1)

## 1.2 - Data Processing

In [ ]:
df.head()

### 1.2.1 - Rename Columns to snake case pattern

In [ ]:
df1 = df.copy()

In [ ]:
cols_new = ['invoice_no','stock_code','description','quantity','invoice_date','unit_price','customer_id','country']
df1.columns = cols_new

### 1.2.2 - Data Dimesions

In [ ]:
print('Number of columns: {}'.format( df1.shape[1] ) )
print('Number of rows: {}'.format( df1.shape[0] ) )

### 1.2.3 - Checking NaN Values

In [ ]:
df1.isna().sum()

#### 1.2.3.1 - Handling NaN Values

In [ ]:
# To analyse the data, two new data set are being created:
df_missing = df1.loc[df1['customer_id'].isna(), :] # Contains the data with missing customer_id
df_not_missing = df1.loc[~df1['customer_id'].isna(),:] # Contain the data with customer_id

In [ ]:
df_not_missing.isna().sum()

In [ ]:
df_missing.isna().sum()

In [ ]:
# Identifying which are the invoices without customer id
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()

# Locate the missing customer_id data by searching from invoice_no
df_aux = df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice)]

# Print result if any
df_aux.head()

In [ ]:
# The analysis will continue with the data set without NaN values
df1 = df_not_missing.copy()

### 1.2.5 - Checkin data types

In [ ]:
df1.dtypes

In [ ]:
df1[ 'invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y') # changing the data on the column invoice data to match the correct data type

df1['customer_id'] = df1['customer_id'].astype( int ) # changing the data on the column customer id data to match the correct data type

df1['country'] = df1['country'].astype( str ) # Changing data type from object to string

df1['quantity'] = df1['quantity'].astype( float ) # Changing data type from object to string

### 1.2.6 - Checking Duplicated Values

There are 5.226 duplicated registers on the data set. Are these a legit acquisition or a duplicated register of one unique sale?

**Assumptions:** I will assume that it is a duplicated register and these will be removed from the analysis.

In [ ]:
n_duplicated = df1.duplicated().sum()
print(f'There are {n_duplicated} duplicated registers.')

In [ ]:
# Counting duplicate occurrences and adding the count as a new feature.
aux = df1.value_counts().reset_index()

# Creation of new dataset containing only the duplicated registers
df_duplicated = aux.loc[aux['count']>1]

In [ ]:
# Calculate the size of the data set with duplicated values
aux_b= len(df1)

# Drop Duplicate
df1 = df1.drop_duplicates().reset_index()

# Calculate the size of the data set after remove the duplicated values
aux_a = len(df1)

n_removed = aux_b - aux_a
print(f'Total of {n_removed} removed from the data set')

**COMMENTS:**

Registers with missing descriptions also lack a customer_id;

To address this issue, the following actions could be considered:

1. Since a unique combination of customer_id and invoice_no can include multiple entries (as granularity is defined by stock_code), compare the invoice_no from the dataset with missing values against the complete dataset and fill in the missing customer_id where matches are found;
   
2. Assign a random number to these customers to avoid data loss;

3. Drop entries with NaN values.

I conducted a test based on approach (1), but it was not possible to locate any customer_id and as the goal is to cluster customers, entries without a valid customer_id will be excluded from the analysis.

Consequently, 135.080 registers will be removed, as they lack proper identification.

## 1.3 - Exploratory Data Analysis

| Variable Name | Role       | Type         | Description                                                                                          | Units     |
|---------------|------------|--------------|------------------------------------------------------------------------------------------------------|-----------|
| InvoiceNo     | ID         | Categorical  | A 6-digit integral number uniquely assigned to each transaction. If this code starts with 'C', it indicates a cancellation. | -         |
| StockCode     | ID         | Categorical  | A 5-digit integral number uniquely assigned to each distinct product.                                | -         |
| Description   | Feature    | Categorical  | Product name                                                                                        | -         |
| Quantity      | Feature    | Integer      | The quantities of each product (item) per transaction                                               | -         |
| InvoiceDate   | Feature    | Date         | The day and time when each transaction was generated                                                | -         |
| UnitPrice     | Feature    | Continuous   | Product price per unit                                                                              | Sterling  |
| CustomerID    | Feature    | Categorical  | A 5-digit integral number uniquely assigned to each customer                                        | -         |
| Country       | Feature    | Categorical  | The name of the country where each customer resides                                                 | -         |


In [ ]:
df1.describe().T

In [ ]:
df1.describe(include='object').T

**COMMENTS**

**1. QUANTITY**

Approximately 75% of the 401603 orders contain a maximum of 12 items. The mean and standard deviation appear to be impacted by high values.

**2. UNIT PRICE**

The products appear to be relatively inexpensive, with an average price of 3.47. Additionally, 75% of the 401603 entries have prices below 3.75. The standard deviation is quite high indicating the presence of outliers, also the max value of this feature confirms that, where one item cost nearly 40.000 pounds.

**3. INVOICE NO**

There are 22,190 unique invoice numbers, indicating that the e-commerce platform processed 22,190 transactions during the period from 29/11/2016 to 08/07/2017. Additionally, invoice number 576339 contains 542 entries.

**4. STOCK CODE**

There are 3,684 unique products sold, with product 85123A leading as the top item by transaction volume.

**5. DESCRIPTION**

There are 3,896 unique descriptions compared to 3,684 unique stock codes, suggesting that some products may have multiple descriptions associated with the same stock code.

**6. COUNTRY**

The transactions span 37 different countries, with the majority originating from the UK.

### 1.3.1 - Univariate Analysis

#### 1.3.1.1 - Quantity

**Negative numbers**: These values for quantity refers to orders that have been canceled.

**High Concentration of quantity close to 0**: From the first chart it's possible to see that most of sales are of small quantity of products. And there is a specific quantity that is the preferrably one, where the sum of products sold which that quantity is more that 500.000.
Then, I have created a new data set containing the registers where the quantity range vary from 0 to 20 and then I have identified that most of the sales are  made of 12 items.

**Assumptions**: As the objective is to select a group of customers, the best customers, then I will void all purchases that were cancelled. This filter will be added in the section 2.

In [ ]:
# Ploting a distribution chart for quantity
fig = px.histogram(df1, x="quantity", y='quantity',
                   marginal="violin")
                   #hover_data=df1.columns)

fig.update_layout(title='Quantity Distribution',
                  title_x=0.5,
                  yaxis_title="Quantity",
                  plot_bgcolor='white')
fig.show()

From the chart above we can see that most of the registers has a quantity close to 0. I will then, select data between a smaller interval and analyse it, just because there is one specific quantity amout that come to my attentido due the hight volum of it sum.

In [ ]:
aux = df1.loc[ (df1['quantity'] > 0) & (df1['quantity'] < 20)]

fig = px.histogram(aux, x="quantity", y='quantity',
                   marginal="violin")
                   #hover_data=df1.columns)

fig.update_layout(title='Quantity Distribution',
                  title_x=0.5,
                  yaxis_title="Quantity",
                  plot_bgcolor='white')
fig.show()

In [ ]:
# Looking into the 5 highest register by quantity
df1.nlargest(5, 'quantity')

In [ ]:
# Looking into the 5 smallest register by quantity
df1.nsmallest(5, 'quantity')

#### 1.3.1.2 - Unit Price

Most products are cheap: costing less than 5.000 pounds.

**There are products with high value**: As previously seem, 75% of the products costs less than 4 pounds. However the standard deviation is high and the max value is nealry 40.000. What should be further investigated

**There are products where the prices is equal to 0**: These entry shall be removed from the analysis.

In [ ]:
# Ploting a distribution chart for unit price

fig = px.histogram(df1, x="unit_price", y='unit_price',
                   marginal="violin",
                   hover_data=df1.columns)

fig.update_layout(title='Unit Price Distribution',
                  title_x=0.5,
                  yaxis_title="Quantity",
                  plot_bgcolor='white')
fig.show()

In [ ]:
df1.nlargest(5, 'unit_price')

The order with a high prices refers to a cancelation, I will investigate it to understand if it is an error, the unit_price of this item is correct, etc.

In [ ]:
# Let's find the purchase order that originate the cancelation register

df1[df1['customer_id']==15098]

**Upon reviewing the records for the customer who canceled the item, I observed the following:**

One item has multiple descriptions: In my understanding, this is an error and should be corrected.

Customer 15098 made five transactions related to the same item, with two of these transactions canceled. Below is the customer's transaction history:

| index  | invoice_no | stock_code | description                  | quantity | invoice_date | unit_price | customer_id | country        |
|--------|------------|------------|------------------------------|----------|--------------|------------|-------------|----------------|
| 222670 | 556442     | 22502      | PICNIC BASKET WICKER SMALL   | 60       | 2017-06-08   | 4.95       | 15098       | United Kingdom |
| 222680 | 556444     | 22502      | PICNIC BASKET WICKER 60 PIECES | 60     | 2017-06-08   | 649.50     | 15098       | United Kingdom |
| 222681 | C556445    | M          | Manual                       | -1       | 2017-06-08   | 38970.00   | 15098       | United Kingdom |
| 222682 | 556446     | 22502      | PICNIC BASKET WICKER 60 PIECES | 1      | 2017-06-08   | 649.50     | 15098       | United Kingdom |
| 222692 | C556448    | 22502      | PICNIC BASKET WICKER SMALL   | -60      | 2017-06-08   | 4.95       | 15098       | United Kingdom |


Based on this analysis, I conclude:

Invoice_no 556442 was canceled, and the product was returned through invoice_no C556448.

Invoice_no 556444 was canceled, and the product was returned through the manual entry invoice_no C556445.

The record to be retained for analysis is the one associated with invoice_no 556446.

In [ ]:
df1.nsmallest(5, 'unit_price')

#### 1.3.1.3 - Stock Code

While analysing the data, it come to attention some stcock_code that does not reffers to products. A filter will be added, aiming to:

POST - Remove. It seems to be a delivery price. Need's to be confirmed with the company;

M    - Keep. It seems to be manual insertions and legit acquisition;

C2   - Remove. It seems to be a delivery price. Need's to be confirmed with the company;

D    - Remove. It is a discount applied on some acquisitions;

DOT  - Remove. It seems to be a delivery price. Need's to be confirmed with the company;

CRUK - Remove. It seems to be kind of comission.

PADS - Remove. Need's to be confirmed

BANK CHARGES - Remove. Bank fees

In [ ]:
df1["length_sc"] = df1["stock_code"].str.strip().str.len()

df1["length_sc"].value_counts(normalize=True)

In [ ]:
df1[df1["length_sc"] < 5]["stock_code"].value_counts(normalize=True)

In [ ]:
df1[df1["length_sc"] > 6]["stock_code"].value_counts(normalize=True)

#### 1.3.1.4 - Country

There are 345 registers where it is not possible to identify the country where the customer is from; Howevere these registers will be kept
once still is possible to identify the customer;

In [ ]:
df1['country'].value_counts(normalize=True)

In [ ]:
# How many registers does not specify the country?

dfc_aux = df1[['country', 'customer_id']].groupby('country').count().reset_index()
dfc_aux = dfc_aux.sort_values(by='customer_id', ascending=False).reset_index(drop=True)
dfc_aux[dfc_aux['country'].isin(['Unspecified', 'European Community'])]


In [ ]:
df1.shape

# 2 - DATA FILTERING

As previously observed, there are some data preparation to be done. This process will be done by filtering the unecessary data.

In [ ]:
# Making a copy of the dataframe, just in case something goes wrong I do not need to reset the kernel and run everything again.
df2 = df1.copy() 

In [ ]:
df2.head()

In [ ]:
df2[df2['customer_id']==15098]

In [ ]:
# REMOVING REGISTERS WHERE THE PURCHASE HAVE BEEN RETURNED OR CANCELLED
# Classifying each invoice as cancelled (True) or not (False) 
df2['invoice_cancelled'] = df2['invoice_no'].str.startswith("C") & (df2['quantity']<0)

# Separting two datasets:
df2_canceled = df2[df2['invoice_cancelled']] # invoices cancelled
df2_ncanceled = df2[~df2['invoice_cancelled']] # invoices not cancelled

# Mergin the above two data set
merged_df = df2_ncanceled.merge(df2_canceled, on=['stock_code','unit_price','customer_id'], how='left')

# Applying function to clean the new data set by removing columns etc.
df2 = drop_and_rename_duplicate_columns(merged_df)

# Removing the register from customer 15098 which had some issues with his purchases and after an analysis some manual registers where
# located and this is creating an outlier.
df2 = df2.drop(df2.index[146375]).reset_index(drop=True)

# # --- NUMERICAL ATTRIBUTES ---

# # Filtering products where price is equal to 0
df2 = df2.loc[df2['unit_price'] > 0, :]

# --- CATEGORICAL ATTRIBUTES ---

# Filtering stock_codes that does not reffers to items
df2 = df2[~df2['stock_code'].isin( ['POST','C2','DOT','PADS','BANK CHARGES'] )]

# Description
df2 = df2.drop( columns='description', axis=1)

## 2.1 - Exploratory Data Analysis

I will re-do some analysis, just to confirm that the above filteres are working

In [ ]:
df2.duplicated().sum()

### 2.1.1 - Quantity

In [ ]:
# Ploting a distribution chart for quantity

fig = px.histogram(df2, x="quantity", y='quantity',
                   marginal="violin",
                   hover_data=df2.columns)

fig.update_layout(title='Quantity Distribution',
                  title_x=0.5,
                  yaxis_title="Quantity",
                  plot_bgcolor='white')
fig.show()

In [ ]:
df2.nlargest(5,'quantity')

In [ ]:
df2.nsmallest(5,'quantity')

### 2.1.2 - Unit Price

In [ ]:
# Ploting a distribution chart for unit price

fig = px.histogram(df2, x="unit_price", y='unit_price',
                   marginal="violin",
                   hover_data=df2.columns)

fig.update_layout(title='Unit Price Distribution',
                  title_x=0.5,
                  yaxis_title="Quantity",
                  plot_bgcolor='white')
fig.show()

In [ ]:
df2.nsmallest(5,'unit_price')

### 2.1.3 - Stock Code

In [ ]:
aux = df2.copy()

In [ ]:
aux["length_sc"] = aux["stock_code"].str.strip().str.len()

aux["length_sc"].value_counts(normalize=True)

In [ ]:
df2.shape

# 3 - FEATURE ENGINEERING

I will create some features that will help to get more insights and also to create a cluster of customers.

In [ ]:
# Another copy of the dataset just run from here if something goes wrong ahead
df3 = df2.copy()

In [ ]:
# data reference
df_ref = df3.drop( ['invoice_no','stock_code','quantity','invoice_date','unit_price','country'],axis=1).drop_duplicates(ignore_index=True)

## 3.1 - Gross Revenue

In [ ]:
# Calculus of the monetary value sold
df3.loc[:,'gross_revenue'] = df3.loc[:,'quantity'] * df3.loc[:,'unit_price']
df_sold = df3.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

df_ref = pd.merge( df_ref, df_sold, on='customer_id', how='left')

df_ref.isna().sum()

In [ ]:
df_ref.duplicated().sum()

## 3.2 - Recency - Day from last purchase

In [ ]:
# Recency
df_recency = df3.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df3['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left')

df_ref.isna().sum()

In [ ]:
df_ref.head()

## 3.3 - Qty of purchases

In [ ]:
df_freq = (df3.loc[:, ['customer_id', 'invoice_no']].drop_duplicates()
                                                    .groupby('customer_id')
                                                    .count()
                                                    .reset_index()
                                                    .rename( columns={'invoice_no':'qty_invoices'}) )
                
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

df_ref.isna().sum()

In [ ]:
df_ref.head()

## 3.4 Qty of products purchased

In [ ]:
# Number of products
df_freqp = (df3.loc[:,['customer_id', 'quantity']].groupby('customer_id')
                                                   .sum().reset_index()
                                                   .rename( columns={'quantity':'qty_prod_purchased'}) )

df_ref = pd.merge(df_ref, df_freqp, on='customer_id', how='left')
df_ref.isna().sum()

In [ ]:
df_ref.head()

## 3.5 - Range of Products per Customer

In [ ]:
df_prod = ( df3.loc[:,['customer_id', 'stock_code']].groupby('customer_id')
                                                    .count()
                                                    .reset_index()
                                                    .rename( columns={'stock_code':'range_of_products'}) )
            
df_ref = pd.merge(df_ref, df_prod, on='customer_id', how='left')
df_ref.isna().sum()

## 3.6 - Average Ticket Value

In [ ]:
# Avg Ticket
df_avg_ticket = ( df3.loc[:, ['customer_id','gross_revenue']].groupby('customer_id')
                                                             .mean()
                                                             .reset_index()
                                                             .rename( columns={'gross_revenue':'avg_ticket'}) )

df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
df_ref.isna().sum()

## 3.7 - Frequency of Purchases

In [ ]:
df2_max = df3[['customer_id','invoice_date']].drop_duplicates().groupby('customer_id').max().reset_index() # finding the last date of purchase per customer
df2_min = df3[['customer_id','invoice_date']].drop_duplicates().groupby('customer_id').min().reset_index() # finding the last date of purchase per customer

df_aux = ( df3[['customer_id','invoice_no','invoice_date']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .agg( max_ =('invoice_date', 'max'),
                                                                 min_ =('invoice_date', 'min'),
                                                                 days_=('invoice_date', lambda x:( (x.max() - x.min() ).days)+1),
                                                                 buy_ =('invoice_no', 'count'))).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id','frequency']], on='customer_id', how='left')

df_ref.isna().sum()

## 3.8 - Invoice Canceled

In [ ]:
# Number of returns
df_returns = df2_canceled[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename( columns={'quantity':'qty_returns'})
df_returns['qty_returns'] = df_returns['qty_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, on='customer_id', how='left')
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0

df_ref.isna().sum()

## 3.9 - Qty avg of producst per customer

In [ ]:
df_ref['avg_qty_products_purchased'] = df_ref['qty_prod_purchased'] / df_ref['qty_invoices']
df_ref.isna().sum()

## 3.10 - Week day most frequent per customer

The day of the week with Monday=0, Sunday=6.

In [ ]:
# Retrieving the week day for the specific date
df3['week_day']= df3['invoice_date'].dt.dayofweek

# Creating the dataframe with day_week per invoice_no
aux_02 = df3[['invoice_no','customer_id','week_day']].drop_duplicates(ignore_index=True)

# Calculus of week day most frequent
aux_03 = aux_02[['customer_id', 'week_day']].groupby('customer_id').apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)

# Adding the new feature into the data set
df_ref = pd.merge( df_ref, aux_03, on='customer_id', how='left')

df_ref.isna().sum()

## 3.11 - Month most frequent per customer

In [ ]:
# Retrieving the week day for the specific date
df3['month']= df3['invoice_date'].dt.month

# Creating the dataframe with day_week per invoice_no
aux_312 = df3[['invoice_no','customer_id','month']].drop_duplicates(ignore_index=True)

# Calculus of week day most frequent
aux_3123 = aux_312[['customer_id', 'month']].groupby('customer_id').apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)

# Adding the new feature into the data set
df_ref = pd.merge( df_ref, aux_3123, on='customer_id', how='left')

df_ref.isna().sum()

## 3.12 - Year week per customer

In [ ]:
# Retrieving the week day for the specific date
df3['year_week'] = df3['invoice_date'].apply(lambda x: x.isocalendar()[1])

In [ ]:
df3.shape

# 4 - EDA

Now, the data is have been prepared and it is ready for an exploratory data analysis. Let's get some insights!

In [ ]:
df4 = df_ref.copy()

In [ ]:
# df4 = df_ref.dropna()
# df4.isna().sum()

## 4.1 - Univariate Analysis

Metrics to be checked out:

1. Cluster coesos - separados
2. Metrics:
    - Min, Max, Range (dispersion)
    - Mean and Median
    - Standard Deviation and Variance
    - CV (Coefficient of Variation)
    - Distribuition

In [ ]:
df4.head()

In [ ]:
df4.describe().T

In [ ]:
# prof = ProfileReport(df4)
# prof.to_file('data_descriptive.html') 

## 4.2 - Bivariate Analysis

In [ ]:
# cols = ['customer_id']
# df42 = df4.drop( cols, axis=1 )

In [ ]:
# plt.figure( figsize=(12, 15) )
# sns.pairplot( df4 )

### Who are the 20 customers that have spent the most money with the company?

In [ ]:
# Group customers by 'customer_id' and calculate the total sum of 'gross_revenue.
aux = df4.groupby('customer_id')['gross_revenue'].sum().reset_index()

# Sort values in descending order by 'gross_revenue.'
aux = aux.sort_values(by='gross_revenue', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['customer_id'].astype(str), 
           y=aux['gross_revenue'],
           marker={'color': aux['gross_revenue'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='£%{y:.3s}', textposition='outside', cliponaxis=False)
fig.update_layout(title='Customer With Highest Total Purchase Amount',
                  title_x=0.5,
                  xaxis_title="Customer",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Who are the 20 customers that made the most purchases?

In [ ]:
# Group 'customer_id' and sort values in descending order by 'invoice_no'
aux = df4[['customer_id','qty_invoices']].sort_values(by='qty_invoices', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['customer_id'].astype(str), 
           y=aux['qty_invoices'],
           marker={'color': aux['qty_invoices'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='Customer With Highest Qty of Invoices',
                  title_x=0.5,
                  xaxis_title="Customer",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Who are the 20 customers with the highest number of returns or canceled orders?

In [ ]:
# Group data by 'customer_id' and calculate the total sum of 'invoice_no'
aux = df2_ncanceled[['invoice_no','customer_id']].drop_duplicates()
aux01 = aux[['customer_id','invoice_no']].groupby('customer_id').count().reset_index()

# Sort values in decending order by qty of invoice no.
aux01 = aux01.sort_values(by='invoice_no', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux01['customer_id'].astype(str), 
           y=aux01['invoice_no'],
           marker={'color': aux01['invoice_no'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='Customer With Highest Qty of Invoices Cancelled',
                  title_x=0.5,
                  xaxis_title="Customer",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### What are the 20 worst-selling items?

In [ ]:
# Group data by stock_code and calculate the total sum of quantity
aux = df2[['stock_code','quantity']].groupby('stock_code').sum().reset_index()

# Sort values in ascending order by quantity
aux = aux.sort_values(by='quantity', ascending=True).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['stock_code'].astype(str), 
           y=aux['quantity'],
           marker={'color': aux['quantity'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='20 Worst-selling Items',
                  title_x=0.5,
                  xaxis_title="Stock Code",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### What are the 20 best-sellings items?

In [ ]:
# Group
aux = df2[['stock_code','quantity']].groupby('stock_code').sum().reset_index()
aux = aux.sort_values(by='quantity', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['stock_code'].astype(str), 
           y=aux['quantity'],
           marker={'color': aux['quantity'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='20 Best-selling Items',
                  title_x=0.5,
                  xaxis_title="Stock Code",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Which country have sold more by quantity of invoices?

In [ ]:
# Dataset containing unique invoice no
aux = df2.drop_duplicates(subset='invoice_no')
aux = aux[['country','invoice_no']].groupby('country').count().reset_index()
aux = aux.sort_values(by='invoice_no', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['country'].astype(str), 
           y=aux['invoice_no'],
           marker={'color': aux['invoice_no'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='Top Countries by qty of invoices',
                  title_x=0.5,
                  xaxis_title="Stock Code",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Which country have sold more by revenue?

In [ ]:
# Calculus of the data
aux = df3[['country','gross_revenue']].groupby('country').sum().reset_index()
aux = aux[['country','gross_revenue']].sort_values(by='gross_revenue', ascending=False).head(20)

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['country'].astype(str), 
           y=aux['gross_revenue'],
           marker={'color': aux['gross_revenue'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='£%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='20 Best-selling Countries by Revenue',
                  title_x=0.5,
                  xaxis_title="Stock Code",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Which week day has the highest sales?

In [ ]:
# The day of the week with Monday=0, Sunday=6.
# Group data by week_day and calculate the total sum of gross_revenue
aux = df3[['week_day','gross_revenue']].groupby('week_day').sum().reset_index()

# Graph Definition
fig = go.Figure(data=[
    go.Bar(x=aux['week_day'].astype(str), 
           y=aux['gross_revenue'],
           marker={'color': aux['gross_revenue'],
                   'colorscale': 'viridis'})])

# DESIGN
fig.update_traces(texttemplate='£%{y:.3s}',textposition='outside', cliponaxis=False)
fig.update_layout(title='Sum of Gross Revenue by Day of the Week',
                  title_x=0.5,
                  xaxis_title="Stock Code",
                  yaxis_title="Amount Sum",
                  plot_bgcolor='white')
fig.show()

### Which month has the highest sales?

In [ ]:
# Retrieve the year and month of each register
aux = df3.copy()
aux['y_month'] = aux['invoice_date'].dt.strftime('%Y-%m')

# Group data by year_month and calculate the total sum of gross_revenue
aux01 = aux[['y_month','gross_revenue']].groupby('y_month')['gross_revenue'].sum().reset_index()

# Graph Definition
fig = px.line(aux01, x='y_month', y='gross_revenue', title='Gross Revenue Over Month')

# Graph Design
fig.update_traces(mode='lines',  # Adiciona marcadores aos pontos de dados
                  texttemplate='£%{y:.2f}',  # Formatação de texto para os valores
                  textposition='top center')  # Posição do texto

# Update Layout
fig.update_layout(title_x=0.5,  # Centraliza o título
                  xaxis_title='Month',  # Título do eixo X
                  yaxis_title='Gross Revenue (£)',  # Título do eixo Y
                  plot_bgcolor='white')  # Cor de fundo do gráfico

# Plot Graph
fig.show()

In [ ]:
# Graph Definition
fig = px.line(df3, x='invoice_date', y='gross_revenue', title='Gross Revenue Over Time')

# Graph Design
fig.update_traces(mode='lines',  # Adiciona marcadores aos pontos de dados
                  texttemplate='£%{y:.2f}',  # Formatação de texto para os valores
                  textposition='top center')  # Posição do texto

# Update Chart Layout
fig.update_layout(title_x=0.5,  # Centraliza o título
                  xaxis_title='Invoice Date',  # Título do eixo X
                  yaxis_title='Gross Revenue (£)',  # Título do eixo Y
                  plot_bgcolor='white')  # Cor de fundo do gráfico

# Plot chart
fig.show()